### Imports 👽

In [1]:
import os
import sys
import pandas as pd
import torch

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_classes.architecture import NeuralNetworkArchitecture
from data_classes.enums import OptimizationMethod
from data_classes.experiment import Experiment, SamplingMethod
from data_classes.scenario import Scenario, ScenarioSettings
from data_classes.training_data import InputData
from data_classes.training_config import AdamTrainingConfig, TrainingSettings
from models import SequentialNeuralNetwork

### Setup Scenario and Training Settings 🧪

In [2]:
experiment = Experiment(
    SAMPLING_METHOD=SamplingMethod.SOBOL,
    SCENARIO=Scenario.SUM_SINES
)

scenario_settings = ScenarioSettings(experiment.SCENARIO)

input_data = InputData(scenario_settings.DATA_PATH)

In [3]:
widths = [6,12,24]
depths = [4,8,16]
learning_rates = [0.01,0.001]
lambdas = [1.0e-04,1.0e-05,1.0e-06,1e-07]
training_set_sizes = scenario_settings.TRINING_SET_SIZES

In [4]:
training_settings = []

for width in widths:
    for depth in depths:
        for learning_rate in learning_rates:
            for lambda_ in lambdas:
                for training_set_size in training_set_sizes:
                    nn_arch = NeuralNetworkArchitecture(
                        INPUT_DIM=scenario_settings.INPUT_DIM,
                        OUTPUT_DIM=scenario_settings.OUTPUT_DIM,
                        NUM_HIDDEN_LAYERS=width,
                        DEPTH=depth,
                        ACTIVATION_FUNCTION=torch.nn.Sigmoid
                    )
                    training_config = AdamTrainingConfig(
                        OPTIMIZER=OptimizationMethod.ADAM,
                        LEARNING_RATE=learning_rate,
                        REG_PARAM=lambda_,
                        NUM_EPOCHS=1000
                    )

                    training_settings.append(TrainingSettings(
                        nn_architecture=nn_arch,
                        training_config=training_config,
                        training_set_size=training_set_size
                    ))

### Run Training 🚀

In [5]:
training_results_list = []

output_dir = os.path.abspath(os.path.join('..', 'data', experiment.SCENARIO.value, 'output'))
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, f'{experiment.SAMPLING_METHOD.value}_{training_config.NUM_EPOCHS}epochs_results.csv')

for ts in training_settings:
    nn = SequentialNeuralNetwork(
        net_arch=ts.nn_architecture
    )
    training_data = input_data.get_training_and_test_data(
        sampling_method=experiment.SAMPLING_METHOD,
        training_set_size=ts.training_set_size
    )
    nn.train(settings=ts.training_config, data=training_data)
    training_results_list.append(nn.training_results)
    df_results = pd.DataFrame(training_results_list)

    df_results.sort_values('test_error', inplace=True)
    df_results.reset_index(drop=True, inplace=True)
    df_results.to_csv(csv_path, index=False)


KeyboardInterrupt: 